# Stored Procedures


In [1]:
!docker stop mysql-container
!docker restart mysql-container
!docker ps

mysql-container
mysql-container
CONTAINER ID   IMAGE     COMMAND                  CREATED      STATUS                  PORTS                                                  NAMES
50e33c579e3f   mysql     "docker-entrypoint.s…"   2 days ago   Up Less than a second   0.0.0.0:3306->3306/tcp, :::3306->3306/tcp, 33060/tcp   mysql-container


In [2]:
import mysql.connector as mysql
import sqlalchemy 
import os 
from dotenv import load_dotenv

In [3]:
load_dotenv('/workspaces/IBM-DS-Course/.env')
user=os.getenv('USER')
password=os.getenv('PASSWORD')
host = 'localhost'
port=3306
db = 'PETSALE'



In [4]:
def get_db_connection():
    return mysql.connect(
        host=host,
        user=user,
        password=password,
        port=port,
        database=db
    )

In [5]:
conn = get_db_connection()
cursor = conn.cursor()
create_db =f'CREATE DATABASE IF NOT EXISTS {db}'
cursor.execute(create_db)

In [6]:
sql_script = '/workspaces/IBM-DS-Course/Course 6 Db and SQL /Module 6 /6.1 Views, Stored Procedures, and Transactions/1.2 .sql/PETSALE-CREATE-v2.sql'
with open(sql_script, 'r') as script:
    script_cmd = script.read()

use_db = f'USE {db}'
cursor.execute(use_db)

In [7]:
try:
    for cmd in script_cmd.split(";"):
        if cmd.strip():
            cursor.execute(cmd)
        conn.commit()
        print("Success")
except Exception as e:
    print(f'{cmd} failed because e')
    
cursor.close()
conn.close()

Success
Success
Success
Success


## Stored Procedure: Exercise 1


 create and execute a stored procedure to read data from a table on mysql phpadmin using SQL.

1. You will create a stored procedure routine named RETRIEVE_ALL.


- - This RETRIEVE_ALL routine will contain an SQL query to retrieve all the records from the PETSALE table, so you don't need to write the same query over and over again. You just call the stored procedure routine to execute the query everytime.
- - To create the stored procedure routine, copy the code below and paste it to the textarea of the SQL page. Click Go.

In [8]:
conn = get_db_connection()
cursor = conn.cursor()

drop_procdcr1_sql = '''
   DROP PROCEDURE IF EXISTS RETRIEVE_ALL;
   '''
cursor.execute(drop_procdcr1_sql)

# Create the new procedure
stored_prodcr1 = '''
CREATE PROCEDURE RETRIEVE_ALL()
BEGIN
   SELECT * FROM PETSALE;
END;
'''

cursor.execute(stored_prodcr1)

2. To call the RETRIEVE_ALL routine, open another SQL tab by clicking Open in new Tab


In [9]:
try:
    cursor2 = conn.cursor()

    # Call the stored procedure
    test = '''CALL RETRIEVE_ALL;'''
    cursor2.execute(test)

    # Fetch all results from the stored procedure
    results = cursor2.fetchall()

    # Print the results
    print(*[row for row in results], sep='\n')

    # Clear any remaining results
    cursor2.nextset()

except Exception as err:
    print(f"Error: {err}")

finally:
    # Close the cursor2 and connection
    cursor2.close()
    conn.close()

(1, 'Cat', Decimal('450.09'), datetime.date(2018, 5, 29), 9)
(2, 'Dog', Decimal('666.66'), datetime.date(2018, 6, 1), 3)
(3, 'Parrot', Decimal('50.00'), datetime.date(2018, 6, 4), 2)
(4, 'Hamster', Decimal('60.60'), datetime.date(2018, 6, 11), 6)
(5, 'Goldfish', Decimal('48.48'), datetime.date(2018, 6, 14), 24)
Error: 2053: Attempt to read a row while there is no result set associated with the statement


3. You can view the created stored procedure routine RETRIEVE_ALL. On the left panel, expand the PETS database option and click on Procedures to view the procedure.


4. If you wish to drop the stored procedure routine RETRIEVE_ALL, copy the code below and paste it to the textarea of the SQL page. Click Go.

In [10]:
conn = get_db_connection() 
cursor = conn.cursor()
drop_procdr1 = '''DROP PROCEDURE RETRIEVE_ALL;
'''
faulty_call = '''CALL RETRIEVE_ALL;'''

cursor.execute(drop_procdr1)
# cursor.execute(faulty_call)
# err_rsp = cursor.fetchall()
# print(*[row for row in err_rsp])# output is ProgrammingError: 1305 (42000): PROCEDURE PETSALE.RETRIEVE_ALL does not exist

cursor.close()

True

## Stored Procedure: Exercise 2


In this exercise, you will create and execute a stored procedure to write/modify data in a table on MySQL using SQL.

You will create a stored procedure routine named UPDATE_SALEPRICE with parameters Animal_ID and Animal_Health.


- This UPDATE_SALEPRICE routine will contain SQL queries to update the sale price of the animals in the PETSALE table depending on their health conditions, BAD or WORSE.

- This procedure routine will take animal ID and health conditon as parameters which will be used to update the sale price of animal in the PETSALE table by an amount depending on their health condition. Suppose that:

- - For animal with ID XX having BAD health condition, the sale price will be reduced further by 25%.
- - For animal with ID YY having WORSE health condition, the sale price will be reduced further by 50%.
- - For animal with ID ZZ having other health condition, the sale price won't change.
- To create the stored procedure routine, copy the code below and paste it to the textarea of the SQL page. Click Go.

In [11]:
cursor = conn.cursor()


In [12]:

drop_stored_procdr2 = '''DROP PROCEDURE IF EXISTS UPDATE_SALEPRICE'''
cursor.execute(drop_stored_procdr2)
create_procdr2 = '''
CREATE PROCEDURE UPDATE_SALEPRICE (IN Animal_ID INTEGER, IN Animal_Health VARCHAR(5))
BEGIN
    IF Animal_Health = 'BAD' THEN
        UPDATE PETSALE
        SET SALEPRICE = SALEPRICE - (SALEPRICE * 0.25)
        WHERE ID = Animal_ID;
    ELSEIF Animal_Health = 'WORSE' THEN
        UPDATE PETSALE
        SET SALEPRICE = SALEPRICE - (SALEPRICE * 0.5)
        WHERE ID = Animal_ID;
    ELSE
        UPDATE PETSALE
        SET SALEPRICE = SALEPRICE
        WHERE ID = Animal_ID;
    END IF;
END;'''
cursor.execute(create_procdr2)


1. Let's call the UPDATE_SALEPRICE routine. We want to update the sale price of animal with ID 1 having BAD health condition in the PETSALE table. open another SQL tab by clicking Open in new Tab

- *Note* if you have dropped RETREIVE_ALL procedure rerun the creation script of that procedure before executing these lines.

In [13]:
drop_procdcr1_sql = '''
DROP PROCEDURE IF EXISTS RETRIEVE_ALL;
'''
cursor.execute(drop_procdcr1_sql)

    # Create the new procedure
stored_prodcr1 = '''
CREATE PROCEDURE RETRIEVE_ALL()
BEGIN
   SELECT * FROM PETSALE;
END;
'''

cursor.execute(stored_prodcr1)
cursor.close()

True

In [23]:
conn.close()

In [24]:
conn = get_db_connection()
cursor = conn.cursor()
q= "CALL RETRIEVE_ALL;"
cursor.execute(q)
rows = cursor.fetchall()
print(*[row for row in rows], sep ='\n')
cursor.close()
conn.close()

(1, 'Cat', Decimal('450.09'), datetime.date(2018, 5, 29), 9)
(2, 'Dog', Decimal('666.66'), datetime.date(2018, 6, 1), 3)
(3, 'Parrot', Decimal('50.00'), datetime.date(2018, 6, 4), 2)
(4, 'Hamster', Decimal('60.60'), datetime.date(2018, 6, 11), 6)
(5, 'Goldfish', Decimal('48.48'), datetime.date(2018, 6, 14), 24)


In [25]:
conn = get_db_connection()
cursor = conn.cursor()
q2= "CALL UPDATE_SALEPRICE(1, 'BAD');"
cursor.execute(q2)
rows = cursor.fetchall()
print(*[row for row in rows], sep ='\n')

In [26]:
q3= "CALL RETRIEVE_ALL;"
cursor.execute(q3)
rows = cursor.fetchall()
print(*[row for row in rows], sep ='\n')
conn.close()

(1, 'Cat', Decimal('337.57'), datetime.date(2018, 5, 29), 9)
(2, 'Dog', Decimal('666.66'), datetime.date(2018, 6, 1), 3)
(3, 'Parrot', Decimal('50.00'), datetime.date(2018, 6, 4), 2)
(4, 'Hamster', Decimal('60.60'), datetime.date(2018, 6, 11), 6)
(5, 'Goldfish', Decimal('48.48'), datetime.date(2018, 6, 14), 24)


2. Let's call the UPDATE_SALEPRICE routine once again. 
- We want to update the sale price of animal with ID 3 having WORSE health condition in the PETSALE table. 

- copy the code below and paste it to the textarea of the SQL page. Click Go. You will have all the records retrieved from the PETSALE table.


In [27]:
conn = get_db_connection()
cursor = conn.cursor()
q2= "CALL UPDATE_SALEPRICE(1, 'BAD');"
cursor.execute(q2)
rows = cursor.fetchall()

q4= "CALL UPDATE_SALEPRICE(3, 'WORSE');"
cursor.execute(q4)
rows = cursor.fetchall()

q5 = "CALL RETRIEVE_ALL;"
cursor.execute(q5)
rows = cursor.fetchall()
print(*[row for row in rows], sep ='\n')

(1, 'Cat', Decimal('337.57'), datetime.date(2018, 5, 29), 9)
(2, 'Dog', Decimal('666.66'), datetime.date(2018, 6, 1), 3)
(3, 'Parrot', Decimal('25.00'), datetime.date(2018, 6, 4), 2)
(4, 'Hamster', Decimal('60.60'), datetime.date(2018, 6, 11), 6)
(5, 'Goldfish', Decimal('48.48'), datetime.date(2018, 6, 14), 24)


3. You can view the created stored procedure routine UPDATE_SALEPRICE. Click on the Routines and view the procedure.


In [30]:
conn.close()

4. If you wish to drop the stored procedure routine UPDATE_SALEPRICE, copy the code below and paste it to the textarea of the SQL page. Click Go.


In [31]:
conn = get_db_connection()
cursor=conn.cursor()
drop_procdr2 = '''DROP PROCEDURE UPDATE_SALEPRICE;'''
cursor.execute(drop_procdr2)
cursor.close()
# CALL UPDATE_SALEPRICE;

True

In [32]:
conn.commit()
conn.close()